<a href="https://colab.research.google.com/github/Murarin8877/FinancialBigDataAnalysis-homework/blob/main/week10%E4%BD%9C%E6%A5%AD_%E9%87%91%E8%9E%8D%E6%96%87%E6%9C%AC%E6%83%85%E7%B7%92%E8%BE%A8%E8%AD%98Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#使用 Hugging Face 框架建立 BERT 模型，針對金融領域文本進行情緒分析訓練。
可使用以下提供的資料集，或選擇其他適合的金融文本資料集。
資料集網址：https://huggingface.co/datasets/takala/financial_phrasebank

In [1]:
pip install transformers datasets torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, ClassLabel
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [3]:
# prompt: Couldn't find any data file at /content/https:/huggingface.co/datasets/takala/financial_phrasebank.

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, ClassLabel
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# #使用 Hugging Face 框架建立 BERT 模型，針對金融領域文本進行情緒分析訓練。
# 可使用以下提供的資料集，或選擇其他適合的金融文本資料集。
# 資料集網址：https://huggingface.co/datasets/takala/financial_phrasebank
# pip install transformers datasets torch scikit-learn

# 加载金融领域情绪数据集
dataset = load_dataset('financial_phrasebank', 'sentences_allagree',split="train")

# 查看数据结构
print(dataset[0])  # 输出样例：{'sentence': '...text...', 'label': 1}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

financial_phrasebank.py:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

The repository for financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

{'sentence': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .', 'label': 1}


In [4]:
# 加载预训练的 BERT Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 定义数据处理函数
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)

# 应用数据预处理
encoded_dataset = dataset.map(preprocess_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

In [5]:
# 划分训练集和测试集
encoded_dataset = encoded_dataset.train_test_split(test_size=0.2)

# 设置数据集格式
encoded_dataset = encoded_dataset.map(lambda x: {'labels': x['label']})
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

train_dataset = encoded_dataset['train']
test_dataset = encoded_dataset['test']


Map:   0%|          | 0/1811 [00:00<?, ? examples/s]

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

In [6]:
# 加载预训练 BERT 模型
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)  # 3个情绪类别


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# 评估指标
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


In [8]:
training_args = TrainingArguments(
    output_dir="./results",            # 输出目录
    evaluation_strategy="epoch",      # 每个epoch评估
    learning_rate=2e-5,               # 学习率
    per_device_train_batch_size=16,   # 训练批次大小
    per_device_eval_batch_size=16,    # 验证批次大小
    num_train_epochs=5,               # 训练轮数
    weight_decay=0.01,                # 权重衰减
    logging_dir="./logs",             # 日志目录
    logging_steps=10,                 # 日志记录步数
    save_strategy="epoch",            # 保存策略
    load_best_model_at_end=True       # 保存最佳模型
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer = Trainer(
    model=model,                         # 模型
    args=training_args,                  # 训练参数
    train_dataset=train_dataset,         # 训练数据集
    eval_dataset=test_dataset,           # 验证数据集
    tokenizer=tokenizer,                 # Tokenizer
    compute_metrics=compute_metrics      # 评估函数
)

# 开始训练
trainer.train()


<ipython-input-9-7f987e690225>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.325800,0.244025,0.918322,0.918717,0.918322,0.916999
2,0.112300,0.112892,0.973510,0.973873,0.973510,0.973545
3,0.059500,0.130127,0.966887,0.969993,0.966887,0.967160
4,0.028200,0.120376,0.964680,0.965954,0.964680,0.964748
5,0.002600,0.120711,0.971302,0.971929,0.971302,0.971375


TrainOutput(global_step=570, training_loss=0.15072681767898694, metrics={'train_runtime': 283.0943, 'train_samples_per_second': 31.986, 'train_steps_per_second': 2.013, 'total_flos': 595622999381760.0, 'train_loss': 0.15072681767898694, 'epoch': 5.0})

In [10]:
# 在测试集上评估模型
results = trainer.evaluate(test_dataset)
print(results)


{'eval_loss': 0.11289213597774506, 'eval_accuracy': 0.9735099337748344, 'eval_precision': 0.9738727799583059, 'eval_recall': 0.9735099337748344, 'eval_f1': 0.9735452146477912, 'eval_runtime': 3.2354, 'eval_samples_per_second': 140.012, 'eval_steps_per_second': 8.963, 'epoch': 5.0}


In [11]:
# 保存训练后的模型
trainer.save_model("./financial_sentiment_model")
tokenizer.save_pretrained("./financial_sentiment_model")


('./financial_sentiment_model/tokenizer_config.json',
 './financial_sentiment_model/special_tokens_map.json',
 './financial_sentiment_model/vocab.txt',
 './financial_sentiment_model/added_tokens.json',
 './financial_sentiment_model/tokenizer.json')

#測試句子

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 加载保存的模型和 Tokenizer
model_path = "./financial_sentiment_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)


In [14]:
import torch

# 定义预测函数
def predict_sentiment(sentence, model, tokenizer):
    # 对句子进行编码
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    model.eval()  # 设置为评估模式
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities


In [15]:
# 定义测试的金融领域句子
test_sentences = [
    "The company's revenue grew significantly this quarter.",  # 正面
    "The stock price plummeted due to poor earnings.",          # 负面
    "The market remains stable despite global uncertainties."   # 中性
]

# 映射情绪标签
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

# 测试每个句子
for sentence in test_sentences:
    predicted_class, probabilities = predict_sentiment(sentence, model, tokenizer)
    print(f"Sentence: {sentence}")
    print(f"Predicted Sentiment: {label_map[predicted_class]}")
    print(f"Probabilities: {probabilities.numpy()}")
    print()


Sentence: The company's revenue grew significantly this quarter.
Predicted Sentiment: Positive
Probabilities: [[0.00577455 0.00553846 0.98868704]]

Sentence: The stock price plummeted due to poor earnings.
Predicted Sentiment: Negative
Probabilities: [[0.98042965 0.00493044 0.01463989]]

Sentence: The market remains stable despite global uncertainties.
Predicted Sentiment: Positive
Probabilities: [[0.06384856 0.01991881 0.9162326 ]]



#忽略

In [17]:
data = [
    {'sentence': 'The stock market is booming.', 'label': 2},
    {'sentence': 'The company faces bankruptcy.', 'label': 0}
]
x = {'sentence': 'The stock market is booming.', 'label': 2}
result = {'labels': x['label']}  # 把 label 改成 labels
print(result)  # {'labels': 2}

# 手動處理第二條數據
x = {'sentence': 'The company faces bankruptcy.', 'label': 0}
result = {'labels': x['label']}
print(result)  # {'labels': 0}

{'labels': 2}
{'labels': 0}


In [24]:
test_sentences = [
       {'sentence': 'The stock market is booming.', 'label': 2},
    {'sentence': 'The company faces bankruptcy.', 'label': 0}
]

def process(x):
  return {'labels': x['label']}

new_data = list(map(process,test_sentences))
print(new_data)

[{'labels': 2}, {'labels': 0}]


In [21]:
square = lambda x: x**2
square(5)

25